<div style="border:solid green 2px; padding: 20px">
Данный проект нацелен исключительно на демонстрацию навыков автора в SQL.

<h1>Анализ сервиса вопросов и ответов о программировании StackOverflow <span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Исследование" data-toc-modified-id="Исследование-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование</a></span><ul class="toc-item"><li><span><a href="#Базовый-анализ" data-toc-modified-id="Базовый-анализ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Базовый анализ</a></span></li><li><span><a href="#Углубленный-анализ" data-toc-modified-id="Углубленный-анализ-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Углубленный анализ</a></span></li></ul></li></ul></div>

## Описание проекта

**Область исследования**:\
База, где хранятся данные о постах за 2008 год, а так же информация о более поздних оценках, которые эти посты получили. 

**Задача**:\
Необходимо, делая запросы к базе, проанализировать тенденции и паттерны в контенте на платформе.

**Структура данных:**
![Схема БД](https://pictures.s3.yandex.net/resources/Frame_353_1_1664969703.png)

**Таблица `stackoverflow.badges`**

Хранит информацию о значках, которые присуждаются за достижения пользователей.
| Поле           | Описание                                           |
|----------------|----------------------------------------------------|
| `id`           | Идентификатор значка, первичный ключ              |
| `name`         | Название значка                                    |
| `user_id`      | Идентификатор пользователя, внешний ключ к `users` |
| `creation_date`| Дата присвоения значка                             |

**Таблица `stackoverflow.post_types`**

Содержит информацию о типе постов.
| Поле | Описание                  |
|------|---------------------------|
| `id` | Идентификатор типа поста  |
| `type` | Тип поста (Question/Answer) |

**Таблица `stackoverflow.posts`**

Содержит информацию о постах.
| Поле                | Описание                                           |
|---------------------|----------------------------------------------------|
| `id`                | Идентификатор поста, первичный ключ               |
| `title`             | Заголовок поста                                    |
| `creation_date`     | Дата создания поста                                |
| `favorites_count`   | Число добавлений в «Закладки»                      |
| `last_activity_date`| Дата последнего действия в посте                  |
| `last_edit_date`    | Дата последнего изменения поста                   |
| `user_id`           | Идентификатор пользователя, внешний ключ к `users`|
| `parent_id`         | Идентификатор поста-вопроса (если это ответ)      |
| `post_type_id`      | Идентификатор типа поста, внешний ключ к `post_types`|
| `score`             | Количество очков, набранных постом                |
| `views_count`       | Количество просмотров                              |

**Таблица `stackoverflow.users`**

Содержит информацию о пользователях.
| Поле              | Описание                                |
|-------------------|-----------------------------------------|
| `id`              | Идентификатор пользователя, первичный ключ |
| `creation_date`   | Дата регистрации пользователя           |
| `display_name`    | Имя пользователя                        |
| `last_access_date`| Дата последнего входа                   |
| `location`        | Местоположение                          |
| `reputation`      | Очки репутации                          |
| `views`           | Число просмотров профиля                |

**Таблица `stackoverflow.vote_types`**

Содержит информацию о типах голосов.
| Поле | Описание                                   |
|------|--------------------------------------------|
| `id` | Идентификатор типа голоса, первичный ключ  |
| `name` | Название метки (UpMod, DownMod, Close и т.д.) |

**Таблица `stackoverflow.votes`**

Содержит информацию о голосах за посты.
| Поле           | Описание                                           |
|----------------|----------------------------------------------------|
| `id`           | Идентификатор голоса, первичный ключ              |
| `post_id`      | Идентификатор поста, внешний ключ к `posts`        |
| `user_id`      | Идентификатор пользователя, внешний ключ к `users` |
| `bounty_amount`| Сумма вознаграждения                               |
| `vote_type_id` | Идентификатор типа голоса, внешний ключ к `vote_types` |
| `creation_date`| Дата назначения голоса                             |

## Исследование

### Базовый анализ

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(*)
FROM stackoverflow.posts
WHERE post_type_id = (
    SELECT id
    FROM stackoverflow.post_types
    WHERE type = 'Question'
)
AND (score > 300 OR favorites_count >= 100);

2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH daily_question_counts AS (
    SELECT 
        DATE(creation_date) AS date,
        COUNT(*) AS question_count
    FROM stackoverflow.posts
    WHERE post_type_id = (
        SELECT id
        FROM stackoverflow.post_types
        WHERE type = 'Question'
    )
    AND creation_date >= '2008-11-01' 
    AND creation_date < '2008-11-19'
    GROUP BY DATE(creation_date)
    ORDER BY DATE(creation_date)
)
SELECT ROUND(AVG(question_count)) AS average_questions_per_day
FROM daily_question_counts;

3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id) AS unique_users_with_badges_on_registration_day
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE DATE(u.creation_date) = DATE(b.creation_date);

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id) AS unique_posts_with_votes
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn';

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT 
    id,
    name,
    RANK() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT 
    v.user_id,
    COUNT(*) AS close_votes_count
FROM stackoverflow.votes v
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY v.user_id
ORDER BY close_votes_count DESC, v.user_id DESC
LIMIT 10;

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:

- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH badge_counts AS (
    SELECT 
        user_id,
        COUNT(*) AS badge_count
    FROM stackoverflow.badges
    WHERE creation_date >= '2008-11-15' AND creation_date < '2008-12-16'
    GROUP BY user_id
),
ranked_badge_counts AS (
    SELECT 
        user_id,
        badge_count,
        DENSE_RANK() OVER (ORDER BY badge_count DESC) AS rank
    FROM badge_counts
)
SELECT 
    user_id,
    badge_count,
    rank
FROM ranked_badge_counts
ORDER BY rank ASC, user_id ASC
LIMIT 10; 

8. Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:

- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT 
    title,
    user_id,
    score,
    ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg_score_per_user
FROM stackoverflow.posts
WHERE title IS NOT NULL
AND score != 0
ORDER BY score ASC

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
WITH user_badge_counts AS (
    SELECT 
        user_id,
        COUNT(*) AS badge_count
    FROM stackoverflow.badges
    GROUP BY user_id
    HAVING COUNT(*) > 1000
)
SELECT 
    p.title
FROM stackoverflow.posts p
JOIN user_badge_counts ubc ON p.user_id = ubc.user_id
WHERE p.title IS NOT NULL;

10. Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу `1`;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу `2`;
- пользователям с числом просмотров меньше 100 — группу `3`.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT 
    id AS user_id,
    views AS profile_views,
    CASE
        WHEN views >= 350 THEN 1
        WHEN views >= 100 THEN 2
        ELSE 3
    END AS user_group
FROM stackoverflow.users
WHERE TRIM(location) LIKE '%Canada%'
  AND views > 0;

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH user_groups AS (
    SELECT 
        id AS user_id,
        views AS profile_views,
        CASE
            WHEN views >= 350 THEN 1
            WHEN views >= 100 THEN 2
            ELSE 3
        END AS user_group
    FROM stackoverflow.users
    WHERE TRIM(location) LIKE '%Canada%'
      AND views > 0
),
max_views_per_group AS (
    SELECT 
        user_group,
        MAX(profile_views) AS max_views
    FROM user_groups
    GROUP BY user_group
)
SELECT 
    ug.user_id,
    ug.user_group,
    ug.profile_views
FROM user_groups ug
JOIN max_views_per_group mvpg ON ug.user_group = mvpg.user_group AND ug.profile_views = mvpg.max_views
ORDER BY ug.profile_views DESC, ug.user_id ASC;

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH daily_registrations AS (
    SELECT 
        EXTRACT(DAY FROM creation_date) AS day,
        COUNT(*) AS daily_user_count
    FROM stackoverflow.users
    WHERE EXTRACT(MONTH FROM creation_date) = 11
      AND EXTRACT(YEAR FROM creation_date) = 2008
    GROUP BY EXTRACT(DAY FROM creation_date)
),
cumulative_registrations AS (
    SELECT 
        day,
        daily_user_count,
        SUM(daily_user_count) OVER (ORDER BY day) AS cumulative_user_count
    FROM daily_registrations
)
SELECT 
    day AS day_number,
    daily_user_count,
    cumulative_user_count
FROM cumulative_registrations
ORDER BY day_number;

13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
WITH first_post_times AS (
    SELECT 
        user_id,
        MIN(creation_date) AS first_post_time
    FROM stackoverflow.posts
    GROUP BY user_id
),
user_registration AS (
    SELECT 
        u.id AS user_id,
        u.creation_date AS registration_date,
        fp.first_post_time
    FROM stackoverflow.users u
    JOIN first_post_times fp ON u.id = fp.user_id
)
SELECT 
    user_id,
    first_post_time - registration_date AS interval_between_registration_and_first_post
FROM user_registration;

### Углубленный анализ

1. Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT 
    CAST(DATE_TRUNC('month', creation_date) AS date) AS month,
    SUM(views_count) AS total_views
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY DATE_TRUNC('month', creation_date)
ORDER BY total_views DESC;

2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT 
    u.display_name,
    COUNT(DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE pt.type = 'Answer'
AND p.creation_date BETWEEN u.creation_date AND u.creation_date + INTERVAL '1 month'
GROUP BY u.display_name
ORDER BY display_name;

3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH active_users AS (
    SELECT u.id AS user_id
    FROM stackoverflow.users u
    JOIN stackoverflow.posts p ON u.id = p.user_id
    WHERE DATE_TRUNC('month', u.creation_date) = '2008-09-01'
      AND DATE_TRUNC('month', p.creation_date) = '2008-12-01'
    GROUP BY u.id
    HAVING COUNT(p.id) > 0
)
SELECT 
    DATE_TRUNC('month', p.creation_date) AS month,
    COUNT(p.id) AS post_count
FROM stackoverflow.posts p
WHERE p.user_id IN (SELECT user_id FROM active_users)
  AND DATE_TRUNC('year', p.creation_date) = '2008-01-01'
GROUP BY DATE_TRUNC('month', p.creation_date)
ORDER BY DATE_TRUNC('month', p.creation_date) DESC;

4. Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT 
    user_id,
    creation_date,
    views_count,
    SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) AS cumulative_views
FROM stackoverflow.posts;

5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
SELECT ROUND(AVG(daily_post_count))
FROM (
    SELECT 
        user_id,
        COUNT(DISTINCT DATE(creation_date)) AS daily_post_count
    FROM stackoverflow.posts
    WHERE DATE(creation_date) BETWEEN '2008-12-01' AND '2008-12-07'
    GROUP BY user_id
) AS subquery;

6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип `numeric`.

In [ ]:
WITH monthly_post_counts AS (
    SELECT 
        EXTRACT(MONTH FROM creation_date::date) AS month,
        COUNT(DISTINCT id) AS post_count
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY EXTRACT(MONTH FROM creation_date::date)
)
SELECT 
    month,
    post_count,
    ROUND(((post_count::numeric / LAG(post_count) OVER (ORDER BY month)) - 1) * 100, 2) AS post_growth
FROM monthly_post_counts;

7. Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:

- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH top_user AS (
    SELECT 
        user_id,
        COUNT(DISTINCT id) AS post_count
    FROM stackoverflow.posts
    GROUP BY user_id
    ORDER BY post_count DESC
    LIMIT 1
),
user_posts_in_october AS (
    SELECT 
        p.user_id,
        p.creation_date,
        EXTRACT(WEEK FROM p.creation_date) AS week_number
    FROM stackoverflow.posts p
    JOIN top_user tu ON p.user_id = tu.user_id
    WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01'
)
SELECT 
    DISTINCT week_number::numeric,
    MAX(creation_date) OVER (PARTITION BY week_number) AS max_creation_date
FROM user_posts_in_october
ORDER BY week_number;